In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import os
import datetime
from dataclasses import dataclass
from typing import List
import tarfile

run_num = 0

In [2]:
cb_palette = sns.color_palette(as_cmap=True)

color_palette = {
    "Expected": cb_palette[0], 
    "expected": cb_palette[0], 
    "woltka": cb_palette[1], 
    "wol": cb_palette[1], 
    "jams": cb_palette[2], 
    "wgsa": cb_palette[3], 
    "wgsa2": cb_palette[3], 
    "biobakery3": cb_palette[4], 
    "bio3": cb_palette[4], 
    "biobakery4": cb_palette[5], 
    "bio4": cb_palette[5]
}

today = datetime.date.today()
date = today.strftime("%Y-%m-%d")

# Gathering and Generating Combined Stats Files

In [3]:
project_root = os.path.abspath("../../pipelines/")
# threshold = 0.0

# First, we load the data from the CSV file.
def find_stats_files(rank: str, threshold: bool):
    for root, dirs, files in os.walk(project_root):
        for file in files:
            # print(file)
            if f"all_stats_replicates_{rank}" in file and file.endswith('.csv'):
                stats_path = os.path.join(root, file)

                df = pd.read_csv(stats_path)

                if threshold:
                    # Add the threshold to the dataframe
                    threshold = file.split("_")[0]
                    if threshold == "all":
                        continue
                    elif threshold == "0.1":
                        continue
                    else:   
                        df["threshold"] = threshold
                
                yield stats_path, df

In [4]:
def combine_stats(rank: str, threshold_files: bool) -> pd.DataFrame:
    """
    Parameters:
        rank: str
            The rank to combine the stats for.
        threshold_files: bool
            Whether the files are thresholded or not.
    Returns:
        df: pd.DataFrame
            The combined dataframe.

    Combines the stats from all the different pipelines into one dataframe.
    """
    combined_df = pd.DataFrame()

    for path, df in find_stats_files(rank, threshold_files):
        df["Source"] = path.split("/")[-2]
        combined_df = pd.concat([combined_df, df])

    return combined_df

In [5]:
def make_catplot(df: pd.DataFrame, id_var: str, src: str, plot_type: str, pdf_output: PdfPages, title: str):
    melted = df.melt(id_vars=["SampleID", id_var, "Source"], var_name="Metric", value_name="Value").dropna()
    # Cast value column to float
    melted["Value"] = melted["Value"].astype(float)

    ax = sns.catplot(data=melted, x=id_var, y="Value", col="Metric", col_wrap=3, kind=plot_type, sharey=False, palette=color_palette)
    ax.fig.suptitle(title, y=1.05)

    pdf_output.savefig(ax.figure, bbox_inches='tight', dpi=300)
    plt.close(ax.figure)

def plot_stats(df: pd.DataFrame, output_pdf: str):
    pdf_output = PdfPages(output_pdf)
    # display(df.head(30))
    for th, th_df in df.groupby("threshold"):
        for src, df in th_df.groupby("Source"):
            title = f'Summary of Statistics for {src} at {th} Threshold'
            if src == "bmock12" or src == "camisimGI" or src =="nist":
                make_catplot(df, "Pipeline", src, "bar", pdf_output, title)

            else:
                no_average = df.loc[df['SampleID'] != 'Average']
                make_catplot(no_average, "Pipeline", src, "box", pdf_output, title)

    pdf_output.close()

In [6]:
def recode_pipeline(df: pd.DataFrame) -> pd.DataFrame:
    recode_dict = {"wgsa": "wgsa2", "wol": "woltka", "bio4": "biobakery4"}

    df["Pipeline"].replace(recode_dict, inplace=True)


def cleanup_combined_df(df: pd.DataFrame) -> pd.DataFrame:
    # display(df.head(30))
    # If the Pipeline value is empty, then replace it with the value in "Source/Pipeline".
    try:
        df.loc[df["Pipeline"].isna(), "Pipeline"] = df.loc[df["Pipeline"].isna(), "Source/Pipeline"]
        # Drop the "Source/Pipeline" column.
        df = df.drop(columns=["Source/Pipeline"])
    except KeyError:
        print("Warning: No Source/Pipeline column found. Skipping...")
        pass

    # Drop rows where the SampleID is "Average".
    df = df.loc[df["SampleID"] != "Average"]

    # Recode the pipeline names.
    recode_pipeline(df)

    return df

In [7]:
# plot_stats(combine_stats("genus"), "stats_summary_genus.pdf")

def make_stats():
    # Check if today's files exist. If they do, we want to delete them because we are going to append to them.
    today_genus = f"results/all_stats_genus_{today}.csv"
    today_species = f"results/all_stats_species_{today}.csv"

    if os.path.exists(today_genus):
        os.remove(today_genus)
    if os.path.exists(today_species):
        os.remove(today_species)
    
    combined_df = cleanup_combined_df(combine_stats("genus", True))
    combined_df.to_csv(today_genus, index=False, mode='a')

    combined_df = cleanup_combined_df(combine_stats("species", True))
    combined_df.to_csv(today_species, index=False, mode='a')

make_stats()
# plot_stats(combined_df, "stats_summary_species.pdf")
# display(combined_df.head(30))

# Subset Plotting with Simple Bars

In [8]:
@dataclass
class Subset:
    pipelines: List[str]
    thresholds: List[float]
    sources: List[str]
    metrics: List[str]

# Only gather stats of interest... no longer using jams202212 as of 2023-03-01.
replicates = Subset(
    pipelines = ["biobakery3", "biobakery4", "jams", "wgsa2", "woltka"],
    thresholds = [0.0001],
    sources = ["mixed", "hilo", "tourlousse"],
    metrics = ["AD", "Sens", "FPRA"],
)

one_to_one = Subset(
    pipelines = ["biobakery3", "biobakery4", "jams", "wgsa2", "woltka"],
    thresholds = [0.0001],
    sources = ["bmock12", "camisimGI", "nist"],
    metrics = ["AD", "Sens", "FPRA"],
)

rename_pipeline_dict = {
    "biobakery3": "Biobakery3", 
    "biobakery4": "Biobakery4", 
    "jams": "JAMS", 
    "wgsa2": "WGSA2", 
    "woltka": "Woltka"
}

rename_source_dict = {
    "mixed": "Amos Mixed",
    "hilo": "Amos HiLo",
    "tourlousse": "Tourlousse",
}

In [9]:
# Open the stats files:
stats_files = [f"results/all_stats_species_{today}.csv", f"results/all_stats_genus_{today}.csv"]

def subset_df(sub: Subset, save: bool = False):
    for sf in stats_files:
        df = pd.read_csv(sf)
        # display(df)
        df = df.loc[df["Pipeline"].isin(sub.pipelines)]
        df = df.loc[df["threshold"].isin(sub.thresholds)]
        df = df.loc[df["Source"].isin(sub.sources)]
        df["Run"] = run_num

        if save:
            df.to_csv(f"{sf.split('.')[0]}_subset.csv", index=False)
        
        rank = sf.split("_")[2].split(".")[0]
    
        yield df, rank


def plot_subset(sub: Subset, tag: str, pdf_output: PdfPages):
    for df, rank in subset_df(sub, save=True):
        id_vars = ["SampleID", "Pipeline", "Source"] + sub.metrics
        df = df[["SampleID", "Pipeline", "Source", "AD", "Sens", "FPRA"]]
        # display(df)
        df = df.melt(id_vars=["SampleID", "Pipeline", "Source"], var_name="Metrics", value_vars=["AD", "Sens", "FPRA"])

        # Rename the pipelines.
        df["Pipeline"].replace(rename_pipeline_dict, inplace=True)

        # We need to make a figure object no bigger than 8.5x11 inches.
        if tag == "One-to-One":
            # df["Community"] = df["SampleID"] + "_" + df["Source"]
            # g = sns.catplot(data=df, kind="bar", x="Pipeline", col="Metrics", row="Source", y="value", hue="Community")

            for src, src_df in df.groupby("Source"):
                print(src)
                g = sns.catplot(data=src_df, kind="bar", x="Pipeline", col="Metrics", y="value", hue="SampleID", sharey=True)
                # Change y-axis label to "Assessment Metric"
                g.set(ylabel="Assessment Metric")
                g.fig.suptitle(f"Summary of {rank.capitalize()} Stats for {src} Community", y=1.025)

                # Add data labels
                # for ax in g.axes.flat:
                #     for p in ax.patches:
                #         ax.annotate(f"{p.get_height():.0f}", (p.get_x() + p.get_width() / 2., p.get_height()),
                #             ha='center', va='center', xytext=(0, 10), textcoords='offset points')

                g.fig.savefig(f"results/images/{rank}_stats_{src}.png", bbox_inches='tight', dpi=300) 
                pdf_output.savefig(g.fig, bbox_inches='tight', dpi=300)
                plt.close(g.fig)

            # df["Community"] = df["SampleID"] + "_" + df["Source"]
            # g = sns.catplot(data=df, kind="bar", x="Pipeline", col="Metrics", row="Community", y="value", hue="Source") -- original
            # g = sns.catplot(data=df, kind="bar", x="Pipeline", col="Community", y="value", hue="Metrics", col_wrap=3)
            # g = sns.catplot(data=df, kind="bar", x="Pipeline", col="Source", row="SampleID", y="value", hue="Metrics", sharey=False)

        else:
            # Rname the sources.
            df["Source"].replace(rename_source_dict, inplace=True)
            g = sns.catplot(data=df, kind="bar", x="Pipeline", col="Metrics", y="value", hue="Source", sharey=True)


        g.fig.suptitle(f"Summary of {rank.capitalize()} Stats for {tag} Communities", y=1.05)
        g.set(ylabel="Assessment Metric")

        # Add data labels
        # for ax in g.axes.flat:
        #     for p in ax.patches:
        #         ax.annotate(f"{p.get_height():.0f}", (p.get_x() + p.get_width() / 2., p.get_height()),
        #                     ha='center', va='center', xytext=(0, 10), textcoords='offset points', fontsize=6)

        # save to png as well
        g.fig.savefig(f"results/images/{rank}_stats_{tag}.png", bbox_inches='tight', dpi=300) 
        pdf_output.savefig(g.fig, bbox_inches='tight', dpi=300)
        plt.close(g.fig)

In [3]:
def tar_images():
    if len(os.listdir("results/images")) == 0:
        return

    with tarfile.open(f"results/images.tar.gz", "w:gz") as tar:
        tar.add("results/images", arcname=os.path.basename("results/images"))
    
    # Clear the images in the directory
    for f in os.listdir("results/images"):
        os.remove(os.path.join("results/images", f))

In [11]:
pdf_output = PdfPages(f"results/summary_stats_{today}.pdf")
plot_subset(sub=replicates, tag="Replicate", pdf_output=pdf_output)
plot_subset(sub=one_to_one, tag="One-to-One", pdf_output=pdf_output)

# tar_images()

pdf_output.close()

bmock12
camisimGI
nist
bmock12
camisimGI
nist


In [4]:
tar_images()